In [67]:
import os
import sys
sys.path.append('..')

from pyopensus import Opensus

In [69]:
host = 'ftp.datasus.gov.br'

In [35]:
class Opensus:
    '''
    
    '''
    def __init__(self) -> None:
        self.host = 'ftp.datasus.gov.br'
        self.baseftp = FTP(self.host)
        self.baseftp.login()
        self.basepath = '/dissemin/publicos/'
        
        self.sys_included = {'sihsus': self.basepath+'SIHSUS/200801_/Dados/', 
                             'siasus': self.basepath+'SIASUS/200801_/Dados/'}
        
    def listdir(self, origin=None):
        '''
            ...
        '''
        if origin is None:
            self.baseftp.cwd('/dissemin/publicos/')
            self.baseftp.retrlines('LIST')
        else:
            if origin.lower() in self.sys_included.keys():
                self.baseftp.cwd(self.sys_included[origin.lower()])
                self.baseftp.retrlines('LIST')
                
    def retrieve_year(self, dest, origin, uf, year, mode="RD", verbose=False):
        '''
            ...
        '''
        this_year = dt.date.today().year
        if year < 2008 or year > 2023:
            raise Exception() # --
            
        if origin.lower() not in self.sys_included.keys():
            raise Exception() # --
        else:
            self.baseftp.cwd(self.sys_included[origin.lower()])
            
        year_str = f'{year}'[2:]
        filename = f"{mode}{uf.upper()}{year_str}"
        month_lst = ['01', '02', '03', '04', '05', '06', '07',
                     '08', '09', '10', '11', '12']
            
        for cur_month in month_lst:
            filename_ = f'{filename}{cur_month}.dbc'
            if verbose:
                print(f'Download do arquivo {filename_} ...', end='')
                
            with open(os.path.join(dest, filename_), 'wb') as fp:
                self.baseftp.retrbinary(f'RETR {filename_}', fp.write)
                
            if verbose:
                print(' Feito.')
                
        
        

In [70]:
obj = Opensus()

In [37]:
#obj.listdir(origin='SIHSUS')

In [74]:
dest = os.path.join(os.environ['HOMEPATH'], 'Documents', 'data', 'opendatasus', 'sihsus')

for year in range(2012, 2024):
    obj.retrieve_year(dest, 'sihsus', 'ce', year, mode='RD', verbose=True)

Download do arquivo RDCE1201.dbc ... Feito.
Download do arquivo RDCE1202.dbc ... Feito.
Download do arquivo RDCE1203.dbc ... Feito.
Download do arquivo RDCE1204.dbc ... Feito.
Download do arquivo RDCE1205.dbc ... Feito.
Download do arquivo RDCE1206.dbc ... Feito.
Download do arquivo RDCE1207.dbc ... Feito.
Download do arquivo RDCE1208.dbc ... Feito.
Download do arquivo RDCE1209.dbc ... Feito.
Download do arquivo RDCE1210.dbc ... Feito.
Download do arquivo RDCE1211.dbc ... Feito.
Download do arquivo RDCE1212.dbc ... Feito.
Download do arquivo RDCE1301.dbc ... Feito.
Download do arquivo RDCE1302.dbc ... Feito.
Download do arquivo RDCE1303.dbc ... Feito.
Download do arquivo RDCE1304.dbc ... Feito.
Download do arquivo RDCE1305.dbc ... Feito.
Download do arquivo RDCE1306.dbc ... Feito.
Download do arquivo RDCE1307.dbc ... Feito.
Download do arquivo RDCE1308.dbc ... Feito.
Download do arquivo RDCE1309.dbc ... Feito.
Download do arquivo RDCE1310.dbc ... Feito.
Download do arquivo RDCE1311.dbc

error_perm: 550 The system cannot find the file specified. 

In [57]:
obj.baseftp.login()

ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

In [47]:
ftplib.all_errors

(ftplib.Error, OSError, EOFError, ssl.SSLError)

In [54]:
from ftplib import OSError

ImportError: cannot import name 'OSError' from 'ftplib' (C:\Users\higor.monteiro\AppData\Local\anaconda3\envs\sevig\Lib\ftplib.py)

In [60]:
ex = FTP(host)

In [61]:
ex.login()

'230 User logged in.'

In [65]:
obj.baseftp.pwd()

ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto